# GPT-2 Two-Phase Training in Google Colab

Цей notebook дозволяє запускати двофазне навчання GPT-2 в Google Colab.

## Важливі обмеження Colab:
- CPU-only (без GPU оптимізацій)
- Обмежений час сесії (12 годин)
- Обмежене дискове простір
- Рекомендується тільки для тестування, не для повного навчання

## 1. Встановлення залежностей

In [ ]:
# Встановлення PyTorch та залежностей для CPU
!pip install torch==2.9.1 transformers==4.57.3 datasets==4.4.2 PyYAML==6.0.3 tqdm==4.67.1 loguru==0.7.3
!pip install tokenizers==0.22.1 safetensors==0.7.0

print("✅ Залежності встановлено")

## 2. Клонування репозиторію

In [ ]:
# Клонування репозиторію
!git clone https://github.com/your-username/your-repo.git AI_project
%cd AI_project

print("✅ Репозиторій клоновано")

## 3. Підготовка датасетів

In [ ]:
# Створення директорій для датасетів
!mkdir -p datasets

# Завантаження датасетів (приклади)
!wget -O datasets/alpaca.json https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json

# Створення мінімального датасету для тестування
import json
minimal_data = {
  "metadata": {"name": "minimal"},
  "data": [
    {
      "instruction": "What is 2+2?",
      "input": "",
      "output": "4"
    },
    {
      "instruction": "Name a color.",
      "input": "",
      "output": "Blue"
    }
  ]
}

with open('datasets/minimal_test.json', 'w') as f:
  json.dump(minimal_data, f, indent=2)

print("✅ Датасети підготовлено")

## 4. Налаштування конфігурації для Colab

In [ ]:
# Створення конфігурації для Colab (CPU-optimized)
import yaml

colab_config = {
  "model": {
    "vocab_size": 50257,
    "n_embd": 320,
    "n_layer": 6,
    "n_head": 8,
    "n_positions": 256,
    "transformer_cache_dir": "models/pretrained"
  },
  "training": {
    "batch_size": 2,  # Менший batch для Colab
    "epochs": 1,
    "learning_rate": 5e-5,
    "gradient_accumulation_steps": 4,
    "max_grad_norm": 1.0,
    "sanity_interval": 50,
    "optimizer": "adamw",
    "weight_decay": 0.01,
    "warmup_steps": 10,
    "checkpoint_dir": "checkpoints",
    "checkpoint_interval": 100,
    "save_every_steps": 25,
    "keep_checkpoints": 3,
    "auto_resume": False,
    "enable_monitoring": True,
    "monitoring_interval": 10,
    "log_dir": "logs",
    "loss_guard_enabled": True,
    "loss_guard_ema_beta": 0.98,
    "loss_guard_warmup_steps": 50,
    "loss_guard_threshold_ratio": 0.5,
    "loss_guard_patience_steps": 100
  },
  "dataset": {
    "path": "datasets/minimal_test.json",  # Мінімальний датасет для тестування
    "type": "trm",
    "format": "instruction",
    "cache_size": 100,
    "validate_format": True,
    "tokenizer_name": "gpt2",
    "add_special_tokens": True
  },
  "cpu_optimization": {
    "num_threads": 2,  # Обмеження для Colab
    "num_workers": 0,
    "pin_memory": False
  }
}

with open('config/colab_phase2.yaml', 'w') as f:
  yaml.dump(colab_config, f, default_flow_style=False)

print("✅ Конфігурацію для Colab створено")

## 5. Запуск Phase 1 (опціонально для тестування)

In [ ]:
# Запуск Phase 1 (тільки для повного тестування)
# Увімкніть, якщо хочете пройти повний цикл

run_phase1 = False  # Змініть на True для запуску

if run_phase1:
  !python scripts/train_phase1_pretraining.py \
    --config config/phase1_pretraining.yaml
else:
  print("ℹ️ Phase 1 пропущено (використовується для тестування)")

## 6. Створення початкової моделі (якщо Phase 1 пропущено)

In [ ]:
# Створення базової моделі для Phase 2
import torch
from transformers import GPT2LMHeadModel, GPT2Config
import os

if not os.path.exists('checkpoints/phase1'):
  os.makedirs('checkpoints/phase1', exist_ok=True)
  
  # Створення конфігурації
  config = GPT2Config(
      vocab_size=50257,
      n_embd=320,
      n_layer=6,
      n_head=8,
      n_positions=256
  )
  
  # Створення моделі
  model = GPT2LMHeadModel(config)
  
  # Збереження
  torch.save(model.state_dict(), 'checkpoints/phase1/best_model.pt')
  
  print("✅ Базову модель для Phase 2 створено")
else:
  print("ℹ️ Модель Phase 1 вже існує")

## 7. Запуск Phase 2 - Instruction Tuning

In [ ]:
# Запуск Phase 2 з оптимізованою конфігурацією
!python scripts/train_phase2_instruction_tuning.py \
  --config config/colab_phase2.yaml \
  --phase1-model checkpoints/phase1/best_model.pt

## 8. Тестування навченої моделі

In [ ]:
# Тестування моделі
!python scripts/test_model.py \
  --model checkpoints/phase2/best_instruction_model.pt \
  --prompt "What is 2+2?" \
  --max-tokens 20

## 9. Інтерактивний чат з моделлю

In [ ]:
# Запуск інтерактивного чату
!python scripts/chat.py \
  --model checkpoints/phase2/best_instruction_model.pt \
  --max-tokens 50

## 10. Моніторинг ресурсів

In [ ]:
# Моніторинг використання ресурсів
import psutil
import time

def monitor_resources():
    cpu_percent = psutil.cpu_percent()
    memory = psutil.virtual_memory()
    disk = psutil.disk_usage('/')
    
    print(f"🖥️ CPU: {cpu_percent}%")
    print(f"💾 RAM: {memory.percent}% ({memory.used/1024/1024/1024:.1f}GB/{memory.total/1024/1024/1024:.1f}GB)")
    print(f"💿 Disk: {disk.percent}% ({disk.used/1024/1024/1024:.1f}GB/{disk.total/1024/1024/1024:.1f}GB)")

# Моніторинг кожні 30 секунд
for i in range(10):
    print(f"\n--- Check {i+1}/10 ---")
    monitor_resources()
    time.sleep(30)

## 11. Збереження результатів на Google Drive

In [ ]:
# Підключення Google Drive та збереження результатів
from google.colab import drive
import shutil
import os
from datetime import datetime

# Підключення Google Drive
drive.mount('/content/drive')

# Створення папки для результатів
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_dir = f"/content/drive/MyDrive/GPT2_Training_{timestamp}"
os.makedirs(results_dir, exist_ok=True)

# Копіювання важливих файлів
files_to_copy = [
    'checkpoints/',
    'logs/',
    'config/colab_phase2.yaml',
    'requirements.txt'
]

for item in files_to_copy:
    if os.path.exists(item):
        dest = os.path.join(results_dir, os.path.basename(item.rstrip('/')))
        if os.path.isdir(item):
            shutil.copytree(item, dest, dirs_exist_ok=True)
        else:
            shutil.copy2(item, dest)
        print(f"✅ Скопійовано: {item} -> {dest}")

print(f"\n🎯 Результати збережено в: {results_dir}")
print("💾 Ви можете знайти всі checkpoints та логи у вашому Google Drive")

## Важливі поради для Colab:

1. **Час сесії**: Colab обмежує сесії ~12 годин. Збережуйте прогрес.
2. **Ресурси**: Використовуйте менші batch sizes для уникнення перевантаження.
3. **Збереження**: Регулярно копіюйте checkpoints на Google Drive.
4. **GPU**: Цей код оптимізований для CPU. Для GPU потрібно змінити конфігурацію.
5. **Датасети**: Використовуйте менші датасети для швидкого тестування.

## Для повного навчання:

Рекомендується використовувати локальну машину або сервер з:
- Більше CPU ядер
- Більше RAM
- Стабільним живленням
- Можливістю запускати довгі тренування